# NB01: Download & Prepare Large Scale Dataset

> The totality of trip data with coordinates (from 2009 - 2016) will achieve 1B+ scale; however, you can limit how much data you choose to download. 

__Notes:__ 

* Friendly for use in Serverless compute on Databricks
* Uses Databricks built-in types and [ST Geospatial Functions](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-st-geospatial-functions), available in public preview since DBR 17.1
* Uses Databricks [H3 Geospatial Functions](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-h3-geospatial-functions), which have been GA since DBR 11.x

---
__Authors:__
- Hubert Boguski <hubert.boguski@spglobal.com>
- Jeff Jaco <jeff.jaco@spglobal.com>
- Debaprasad Satapathy <debaprasad.satapathy@spglobal.com>
- Michael Johns <mjohns@databricks.com>

## [1] Setup

In [0]:
import os

In [0]:
catalog_name = 'mjohns'            # <- replace with yours (must exist)
schema_name = 'liquid_nyc_h3_trip'
spark.catalog.setCurrentCatalog(catalog_name) 
sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
spark.catalog.setCurrentDatabase(schema_name)

## [2] Download CSV Data

> Data pulled from [NYC Taxi Trip Data](https://data.cityofnewyork.us/browse?Data-Collection_Data-Collection=TLC+Trip+Data&sortBy=alpha&pageSize=20)

For Yellow taxi, there are 1.250 Billion trip records with pickup and dropoff latitudes and longitudes for 2009 to 2016 (2017+ does not include coordinates and CSVs have the coordinates, not the PARQUET files):

* 2016 - 131M
* 2015 - 146M
* 2014 - 165M
* 2013 - 173M
* 2012 - 167M
* 2011 - 135M
* 2010 - 169M
* 2009 - 171M

__Note:__ Out of convenience, we have pulled JAN-MAR 2016 CSV data from [Kaggle](https://www.kaggle.com/datasets/elemento/nyc-yellow-taxi-trip-data). You will need to login to either Kaggle or NYC Open Data and download to your local machine and then upload to your Volume to get started with this example. Let's look at our Volume data.

In [0]:
VOL_DIR = '/Volumes/mjohns/liquid_nyc_h3_trip/data/yellow_2016' # <- replace with yours (must exist)
os.listdir(VOL_DIR)

['yellow_tripdata_2016-01.csv',
 'yellow_tripdata_2016-02.csv',
 'yellow_tripdata_2016-03.csv']

__We have 35M Records from the raw CSV data__

> There are 1.250B records available with coordinates (2009 - 2016), if you want to use the entire dataset.

In [0]:
df_taxi = (
  spark
    .read
    .csv(VOL_DIR, inferSchema=True, header=True)
)
df_taxi.createOrReplaceTempView("yellow_trip")
print(f"count? {df_taxi.count():,}")
df_taxi.limit(1000).display()

count? 34,499,859


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2016-01-01T00:00:00Z,2016-01-01T00:00:00Z,2,1.1,-73.99037170410156,40.73469543457031,1,N,-73.98184204101562,40.73240661621094,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8
2,2016-01-01T00:00:00Z,2016-01-01T00:00:00Z,5,4.9,-73.98078155517578,40.72991180419922,1,N,-73.94447326660156,40.716678619384766,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3
2,2016-01-01T00:00:00Z,2016-01-01T00:00:00Z,1,10.54,-73.98455047607422,40.6795654296875,1,N,-73.95027160644531,40.78892517089844,1,33.0,0.5,0.5,0.0,0.0,0.3,34.3
2,2016-01-01T00:00:00Z,2016-01-01T00:00:00Z,1,4.75,-73.99346923828125,40.718990325927734,1,N,-73.96224212646484,40.65733337402344,2,16.5,0.0,0.5,0.0,0.0,0.3,17.3
2,2016-01-01T00:00:00Z,2016-01-01T00:00:00Z,3,1.76,-73.96062469482422,40.78133010864258,1,N,-73.97726440429688,40.758514404296875,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8
2,2016-01-01T00:00:00Z,2016-01-01T00:18:30Z,2,5.52,-73.98011779785156,40.74304962158203,1,N,-73.91349029541016,40.76314163208008,2,19.0,0.5,0.5,0.0,0.0,0.3,20.3
2,2016-01-01T00:00:00Z,2016-01-01T00:26:45Z,2,7.45,-73.99405670166016,40.71998977661133,1,N,-73.96636199951172,40.78987121582031,2,26.0,0.5,0.5,0.0,0.0,0.3,27.3
1,2016-01-01T00:00:01Z,2016-01-01T00:11:55Z,1,1.2,-73.97942352294922,40.74461364746094,1,N,-73.99203491210938,40.753944396972656,2,9.0,0.5,0.5,0.0,0.0,0.3,10.3
1,2016-01-01T00:00:02Z,2016-01-01T00:11:14Z,1,6.0,-73.94715118408203,40.791046142578125,1,N,-73.92076873779297,40.865577697753906,2,18.0,0.5,0.5,0.0,0.0,0.3,19.3
2,2016-01-01T00:00:02Z,2016-01-01T00:11:08Z,1,3.21,-73.99834442138672,40.72389602661133,1,N,-73.995849609375,40.68840026855469,2,11.5,0.5,0.5,0.0,0.0,0.3,12.8


## [3] Prepare Base Delta Tables

> These tables support the initially explored approach used in the blog.

### Table `taxi_trips`

> This uses Databricks built-in ST_ expressions for creating point geometries and clustering by their geohashes. __Note:__ There are 33.9M valid trips in this data.

In [0]:
%sql 
-- Notice storing both pickups and dropoff in the same table
CREATE OR REPLACE TABLE taxi_trips CLUSTER BY (geohash_pickup, geohash_dropoff) AS (
  SELECT 
    st_geohash(geom_pickup) as geohash_pickup, -- geohash for clustering
    st_geohash(geom_dropoff) as geohash_dropoff, -- geohash for clustering
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) / 60.0 as travel_time_minutes,
    HOUR(pickup_datetime) as pickup_hour,
    DAY(pickup_datetime) as pickup_day,
    MONTH(pickup_datetime) as pickup_month,
    YEAR(pickup_datetime) as pickup_year,
    HOUR(dropoff_datetime) as dropoff_hour,
    DAY(dropoff_datetime) as dropoff_day,
    MONTH(dropoff_datetime) as dropoff_month,
    YEAR(dropoff_datetime) as dropoff_year,
    *
  FROM (
    SELECT
      total_amount,
      tpep_pickup_datetime as pickup_datetime,   -- tpep_* for 2016
      tpep_dropoff_datetime as dropoff_datetime, -- tpep_* for 2016
      trip_distance,
      passenger_count,
      payment_type,
      fare_amount,
      extra,
      mta_tax,
      tip_amount,
      tolls_amount,
      st_point(pickup_longitude, pickup_latitude, 4326) as geom_pickup,    -- native geometry
      st_point(dropoff_longitude, dropoff_latitude, 4326) as geom_dropoff, -- native geometry
      pickup_longitude, 
      pickup_latitude,
      dropoff_longitude,
      dropoff_latitude
    FROM
      yellow_trip
  )
  WHERE 
    YEAR(pickup_datetime) < 2025 -- Source data has up to year 2090
    AND cast(pickup_latitude AS int) <> 0 AND pickup_latitude >= -90 AND pickup_latitude <= 90
    AND cast(pickup_longitude AS int) <> 0 AND pickup_longitude >= -180 AND pickup_longitude <= 180
    AND cast(dropoff_latitude AS int) <> 0 AND dropoff_latitude >= -90 AND dropoff_latitude <= 90
    AND cast(dropoff_longitude AS int) <> 0 AND dropoff_longitude >= -180 AND dropoff_longitude <= 180
);

OPTIMIZE taxi_trips FULL;

path,metrics
s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/d9fff4e6-d2cb-43f6-8051-3ba5426b5602,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 16, 0, false, 0, 0, 1756408662725, 1756408666509, 16, 0, null, List(0, 0), null, 28, 28, 0, 0, List(2258770466, false, false, true, 0.9368818168351241, List(0.7500117587138917, 0.7475406933093576), 1.0, null, 0, 0, 0, 0, 0, 0, 0, null, log, 16777216, 67108864, 4, 0, 0, null, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, List(123, 69, 851, 0, 0, 0), 2, 1, 5, sizeAware))"
s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/d9fff4e6-d2cb-43f6-8051-3ba5426b5602,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 16, 0, true, 0, 0, 1756408666561, 1756408666930, 16, 0, null, List(0, 0), SNAPPY, 28, 28, 0, 0, null)"


In [0]:
%sql
SELECT format_number(count(1),0) as valid_trip_cnt FROM taxi_trips

valid_trip_cnt
"33,921,861"


In [0]:
%sql
SELECT * FROM taxi_trips LIMIT 1000

geohash_pickup,geohash_dropoff,travel_time_minutes,pickup_hour,pickup_day,pickup_month,pickup_year,dropoff_hour,dropoff_day,dropoff_month,dropoff_year,total_amount,pickup_datetime,dropoff_datetime,trip_distance,passenger_count,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,geom_pickup,geom_dropoff,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
dr5ruhrhfhkq,dr72ju89wsd7,20.533333,0,2,1,2016,1,2,1,2016,24.3,2016-01-02T00:48:08Z,2016-01-02T01:08:40Z,6.72,6,1,21.0,0.5,0.5,2.0,0.0,SRID=4326;POINT(-73.99420166015625 40.76148986816406),SRID=4326;POINT(-73.92603302001953 40.806251525878906),-73.99420166015625,40.76148986816406,-73.92603302001953,40.806251525878906
dr5rut1etwn1,dr5rvv5kkr8s,12.283333,1,2,1,2016,1,2,1,2016,17.3,2016-01-02T01:41:14Z,2016-01-02T01:53:31Z,4.11,2,1,14.5,0.5,0.5,1.5,0.0,SRID=4326;POINT(-73.97962951660156 40.765403747558594),SRID=4326;POINT(-73.92235565185547 40.765541076660156),-73.97962951660156,40.765403747558594,-73.92235565185547,40.765541076660156
dr5ruyk1ebm5,dr72j3896u54,6.850000,2,2,1,2016,2,2,1,2016,10.3,2016-01-02T02:50:14Z,2016-01-02T02:57:05Z,1.92,1,1,8.0,0.5,0.5,1.0,0.0,SRID=4326;POINT(-73.96528625488281 40.77189636230469),SRID=4326;POINT(-73.94816589355469 40.78972625732422),-73.96528625488281,40.77189636230469,-73.94816589355469,40.78972625732422
dr5rukmr09qv,dr72mwhpuyx6,15.550000,3,2,1,2016,3,2,1,2016,31.8,2016-01-02T03:25:49Z,2016-01-02T03:41:22Z,8.47,5,1,24.5,0.5,0.5,6.0,0.0,SRID=4326;POINT(-73.98567962646484 40.76185989379883),SRID=4326;POINT(-73.93228149414062 40.859519958496094),-73.98567962646484,40.76185989379883,-73.93228149414062,40.859519958496094
dr5rukgjjz5e,dr5rznj1g58x,19.183333,3,2,1,2016,4,2,1,2016,33.35,2016-01-02T03:52:52Z,2016-01-02T04:12:03Z,9.2,2,1,26.5,0.5,0.5,5.55,0.0,SRID=4326;POINT(-73.9885482788086 40.7642936706543),SRID=4326;POINT(-73.86507415771484 40.77058410644531),-73.9885482788086,40.7642936706543,-73.86507415771484,40.77058410644531
dr5ruu4061uq,dr5rvyj2v748,11.050000,4,2,1,2016,4,2,1,2016,14.3,2016-01-02T04:39:21Z,2016-01-02T04:50:24Z,3.84,5,2,13.0,0.5,0.5,0.0,0.0,SRID=4326;POINT(-73.96810913085938 40.75933074951172),SRID=4326;POINT(-73.91956329345703 40.770408630371094),-73.96810913085938,40.75933074951172,-73.91956329345703,40.770408630371094
dr5rukp5r2wc,dr5x1pm9xm34,28.850000,5,2,1,2016,6,2,1,2016,70.01,2016-01-02T05:38:08Z,2016-01-02T06:06:59Z,18.17,1,1,52.0,0.0,0.5,11.67,5.54,SRID=4326;POINT(-73.98298645019531 40.75983810424805),SRID=4326;POINT(-73.77631378173828 40.64558029174805),-73.98298645019531,40.75983810424805,-73.77631378173828,40.64558029174805
dr5ruw80e5nq,dr5rzn8y589y,20.100000,6,2,1,2016,6,2,1,2016,30.36,2016-01-02T06:34:37Z,2016-01-02T06:54:43Z,7.65,1,1,24.5,0.0,0.5,5.06,0.0,SRID=4326;POINT(-73.98179626464844 40.77311325073242),SRID=4326;POINT(-73.87088775634766 40.774044036865234),-73.98179626464844,40.77311325073242,-73.87088775634766,40.774044036865234
dr5ruqtt05t9,dr72j1kdnkkb,7.583333,6,2,1,2016,6,2,1,2016,12.35,2016-01-02T06:42:45Z,2016-01-02T06:50:20Z,2.7,1,1,9.5,0.0,0.5,2.05,0.0,SRID=4326;POINT(-73.98535919189453 40.773887634277344),SRID=4326;POINT(-73.95350646972656 40.788482666015625),-73.98535919189453,40.773887634277344,-73.95350646972656,40.788482666015625
dr5rvbzxe91f,dr5rxp9tryfm,7.916667,7,2,1,2016,7,2,1,2016,12.8,2016-01-02T07:24:40Z,2016-01-02T07:32:35Z,2.6,1,1,10.0,0.0,0.5,2.0,0.0,SRID=4326;POINT(-73.91654968261719 40.74271774291992),SRID=4326;POINT(-73.86967468261719 40.735496520996094),-73.91654968261719,40.74271774291992,-73.86967468261719,40.735496520996094


__What are the top 10 pickup areas [by geohash]?__

In [0]:
%sql
select geohash_pickup, count(1) as cnt_pickup from taxi_trips group by geohash_pickup order by cnt_pickup desc limit 10

Databricks visualization. Run in Databricks to view.

geohash_pickup,cnt_pickup
dr72jp5ze6kp,1523
dr5rzjy77yvj,497
dr5rzn8u5zv4,256
dr5ryywq065k,250
dr5x1pm2qnpp,240
dr5ryywnpvdq,229
dr5x1pm2wjr4,225
dr5rzn8u7s1f,224
dr5rzn8u7uj4,223
dr5rzn8u5xcf,222


__What are the top 10 dropoff areas?__

In [0]:
%sql
select geohash_dropoff, count(1) as cnt_dropoff from taxi_trips group by geohash_dropoff order by cnt_dropoff desc limit 10

Databricks visualization. Run in Databricks to view.

geohash_dropoff,cnt_dropoff
dr72jp5ze6kp,1523
dr5rzjy77yvj,491
dr5x2c2f6rxs,123
dr5ruh0z9fqx,103
dr5ru4qu8h85,94
dr5ruh0zd6rx,94
dr5ruh2b52h0,93
dr5ruh0z0fhk,93
dr5ru4qdf0uk,92
dr5rzjxmpu62,92


## [4] Prepare H3 Delta Tables

> These tables support the approach used in the blog and are derived from what was generated in [3]. __Note:__ We are generating cellids for points at multiple resolution to support various zoom levels.

In [0]:
%sql
CREATE OR REPLACE TABLE h3_taxi_trips CLUSTER BY (pickup_cell_12, dropoff_cell_12, geohash_pickup, geohash_dropoff) AS (
  SELECT
    h3_longlatash3(pickup_longitude, pickup_latitude, 12) as pickup_cell_12,
    h3_longlatash3(pickup_longitude, pickup_latitude, 10) as pickup_cell_10,
    h3_longlatash3(pickup_longitude, pickup_latitude, 8) as pickup_cell_8,
    h3_longlatash3(dropoff_longitude, dropoff_latitude, 12) as dropoff_cell_12,
    h3_longlatash3(dropoff_longitude, dropoff_latitude, 10) as dropoff_cell_10,
    h3_longlatash3(dropoff_longitude, dropoff_latitude, 8) as dropoff_cell_8,
    *
    FROM taxi_trips
);

OPTIMIZE h3_taxi_trips FULL;

path,metrics
s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/ccdf81e2-63af-465b-a644-ed50952a8e3d,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 24, 0, false, 0, 0, 1756408759411, 1756408761409, 16, 0, null, List(0, 0), null, 34, 32, 0, 0, List(2601836063, false, false, true, 0.9578469205354894, List(0.6658722225071669, 0.5034188440958857, 0.5087089136181407, 0.502599183965095), 1.0, null, 0, 0, 0, 0, 0, 0, 0, null, log, 16777216, 67108864, 4, 0, 0, null, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, List(82, 20, 395, 0, 0, 0), 2, 1, 5, sizeAware))"
s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/ccdf81e2-63af-465b-a644-ed50952a8e3d,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 24, 0, true, 0, 0, 1756408761444, 1756408761675, 16, 0, null, List(0, 0), SNAPPY, 34, 32, 0, 0, null)"


In [0]:
%sql
-- same number of records as base table
SELECT format_number(count(1),0) as valid_trip_cnt FROM h3_taxi_trips

valid_trip_cnt
"33,921,861"


In [0]:
%sql
SELECT * FROM h3_taxi_trips LIMIT 1000

pickup_cell_12,pickup_cell_10,pickup_cell_8,dropoff_cell_12,dropoff_cell_10,dropoff_cell_8,geohash_pickup,geohash_dropoff,travel_time_minutes,pickup_hour,pickup_day,pickup_month,pickup_year,dropoff_hour,dropoff_day,dropoff_month,dropoff_year,total_amount,pickup_datetime,dropoff_datetime,trip_distance,passenger_count,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,geom_pickup,geom_dropoff,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
631243921501001727,622236722246287359,613229522992496639,631243922688351743,622236723433635839,613229524175290367,dr5ruxe5hzp7,dr5rsrf2e42x,17.183333,16,1,1,2016,16,1,1,2016,18.3,2016-01-01T16:03:39Z,2016-01-01T16:20:50Z,3.8,1,1,14.5,0.0,0.5,3.0,0.0,SRID=4326;POINT(-73.97760009765625 40.779056549072266),SRID=4326;POINT(-73.98970031738281 40.73603057861328),-73.97760009765625,40.779056549072266,-73.98970031738281,40.73603057861328
631243921457997311,622236722203262975,613229522950553599,631243922688514559,622236723433799679,613229524179484671,dr5rvn4md9v2,dr5ru076vewp,19.883333,17,2,1,2016,17,2,1,2016,17.3,2016-01-02T17:08:46Z,2016-01-02T17:28:39Z,3.9,1,2,15.5,1.0,0.5,0.0,0.0,SRID=4326;POINT(-73.95675659179688 40.771217346191406),SRID=4326;POINT(-73.99919891357422 40.73917007446289),-73.95675659179688,40.771217346191406,-73.99919891357422,40.73917007446289
631243921501144063,622236722246418431,613229522992496639,631243922685761535,622236723431047167,613229524177387519,dr5ruxuhnv8n,dr5ru0fku2yh,19.833333,19,2,1,2016,19,2,1,2016,18.8,2016-01-02T19:27:48Z,2016-01-02T19:47:38Z,3.2,1,1,15.0,0.0,0.5,3.0,0.0,SRID=4326;POINT(-73.97615051269531 40.78059387207031),SRID=4326;POINT(-74.00062561035156 40.742244720458984),-73.97615051269531,40.78059387207031,-74.00062561035156,40.742244720458984
631243921457962495,622236722203230207,613229522950553599,631243922684077567,622236723429343231,613229524175290367,dr5rvn6cgvjh,dr5rsrx4r15x,20.166667,0,1,1,2016,0,1,1,2016,14.8,2016-01-01T00:28:17Z,2016-01-01T00:48:27Z,3.77,2,2,13.5,0.5,0.5,0.0,0.0,SRID=4326;POINT(-73.95601654052734 40.77196502685547),SRID=4326;POINT(-73.98300170898438 40.73495101928711),-73.95601654052734,40.77196502685547,-73.98300170898438,40.73495101928711
631243921458478591,622236722203754495,613229522950553599,631243922687398399,622236723432685567,613229524179484671,dr5ruypp8q53,dr5rspfbppgd,28.500000,11,29,1,2016,12,29,1,2016,22.3,2016-01-29T11:58:31Z,2016-01-29T12:27:01Z,3.8,1,1,19.0,0.0,0.5,2.5,0.0,SRID=4326;POINT(-73.96131896972656 40.771583557128906),SRID=4326;POINT(-73.99982452392578 40.7359733581543),-73.96131896972656,40.771583557128906,-73.99982452392578,40.7359733581543
631243921463346687,622236722208636927,613229522954747903,631243922681371647,622236723426656255,613229524173193215,dr5ruzjj26em,dr5ru26bce50,11.700000,11,3,1,2016,11,3,1,2016,15.36,2016-01-03T11:35:07Z,2016-01-03T11:46:49Z,3.04,2,1,12.0,0.0,0.5,2.56,0.0,SRID=4326;POINT(-73.96406555175781 40.77667236328125),SRID=4326;POINT(-73.98907470703125 40.73882293701172),-73.96406555175781,40.77667236328125,-73.98907470703125,40.73882293701172
631243921457944575,622236722203230207,613229522950553599,631243922690413055,622236723435700223,613229524181581823,dr5rvn4wbgh2,dr5ru9ynk8ec,10.183333,18,2,1,2016,18,2,1,2016,9.8,2016-01-02T18:07:59Z,2016-01-02T18:18:10Z,1.94,1,2,9.0,0.0,0.5,0.0,0.0,SRID=4326;POINT(-73.95648956298828 40.77143859863281),SRID=4326;POINT(-73.97349548339844 40.74799346923828),-73.95648956298828,40.77143859863281,-73.97349548339844,40.74799346923828
631243922648145407,622236723393429503,613229524139638783,631243922651763711,622236723397033983,613229524143833087,dr5rv1d0p058,dr5ru9r78qg2,13.100000,22,1,1,2016,22,1,1,2016,18.35,2016-01-01T22:16:32Z,2016-01-01T22:29:38Z,4.2,1,1,14.0,0.5,0.5,3.05,0.0,SRID=4326;POINT(-73.9569091796875 40.74554443359375),SRID=4326;POINT(-73.97196197509766 40.744808197021484),-73.9569091796875,40.74554443359375,-73.97196197509766,40.744808197021484
631243921458682879,622236722203885567,613229522950553599,63

__What are the top 10 pickup areas [by cellid]?__

> We will do more h3 map viz in the next notebook.

In [0]:
%sql
select pickup_cell_12, count(1) as cnt_pickup from h3_taxi_trips group by pickup_cell_12 order by cnt_pickup desc limit 10

Databricks visualization. Run in Databricks to view.

pickup_cell_12,cnt_pickup
631243922693901311,63118
631243935024619007,58411
631243923270481407,51637
631243923270407679,51545
631243923274357247,42673
631243923274357759,32648
631243935025025535,31150
631243935025026559,30995
631243923270406655,27300
631243922693227519,26608


__What are the top 10 dropoff areas [by cellid]?__

> We will do more h3 map viz in the next notebook.

In [0]:
%sql
select dropoff_cell_12, count(1) as cnt_dropoff from h3_taxi_trips group by dropoff_cell_12 order by cnt_dropoff desc limit 10

Databricks visualization. Run in Databricks to view.

dropoff_cell_12,cnt_dropoff
631243922693897215,23006
631243922693229567,19238
631243922693229055,18834
631243949842909695,17539
631243922693243903,16538
631243922693248511,15240
631243922693711359,15083
631243922693227519,14631
631243922693871615,14510
631243922693774335,13483
